# Latent Dirichlet Allocation

In [9]:
!pip install nltk
import nltk
import pandas as pd

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

from nltk.corpus import stopwords

from nltk.tokenize import word_tokenize

# stematisation : 
from nltk.stem.porter import PorterStemmer 

# bags of modeling
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

# accurracy 
from sklearn.naive_bayes import MultinomialNB

# gridsearch 
from sklearn.model_selection import GridSearchCV

# pipeline
from sklearn.pipeline import Pipeline

from sklearn.decomposition import LatentDirichletAllocation

import string

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\utilisateur\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\utilisateur\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\utilisateur\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
data = pd.read_csv('data', sep=",", header=None)

data.columns = ['text']

data.head()

,text
0,From: gld@cunixb.cc.columbia.edu (Gary L Dare)...
1,From: atterlep@vela.acs.oakland.edu (Cardinal ...
2,From: miner@kuhub.cc.ukans.edu\nSubject: Re: A...
3,From: atterlep@vela.acs.oakland.edu (Cardinal ...
4,From: vzhivov@superior.carleton.ca (Vladimir Z...


The data is a collection of emails that are not labelled. Let's try extract topics from them!

## Preprocessing 

👇 You're used to it by now... Clean up! Store the cleaned text in a new dataframe column "clean_text".

In [13]:
#  Remove punctuation
string.punctuation
for punctuation in string.punctuation : 
    data['text'] = data['text'].str.replace(punctuation, "").str.lower().str.replace(r'\b\d+\b', '')

C:\Users\utilisateur\AppData\Local\Temp\ipykernel_15580\4134947805.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  data['text'] = data['text'].str.replace(punctuation, "").str.lower().str.replace(r'\b\d+\b', '')
C:\Users\utilisateur\AppData\Local\Temp\ipykernel_15580\4134947805.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data['text'] = data['text'].str.replace(punctuation, "").str.lower().str.replace(r'\b\d+\b', '')


In [14]:
def tokenize(clean_text) :
    """nltk le fait automatiquement : il prend la string et la sépare en mots"""
    return word_tokenize(clean_text)

def remov(clean_text): 
    """prends ma liste de stopwords"""
    stop_words = set(stopwords.words('english'))
# conserve les mots si ils ne sont pas dans la liste du dessus
    words = [w for w in clean_text if w not in stop_words]
# prend une liste de string et la remets en une liste avec tous les mots 
    words = ' '.join(words)
    return words

# remove stopwords
data['clean_text']= data['text'].apply(tokenize)
data['clean_text'] = data['clean_text'].apply(remov)
data

,text,clean_text
0,from gldcunixbcccolumbiaedu gary l dare\nsubje...,gldcunixbcccolumbiaedu gary l dare subject sta...
1,from atterlepvelaacsoaklandedu cardinal ximene...,atterlepvelaacsoaklandedu cardinal ximenez sub...
2,from minerkuhubccukansedu\nsubject re ancient ...,minerkuhubccukansedu subject ancient books org...
3,from atterlepvelaacsoaklandedu cardinal ximene...,atterlepvelaacsoaklandedu cardinal ximenez sub...
4,from vzhivovsuperiorcarletonca vladimir zhivov...,vzhivovsuperiorcarletonca vladimir zhivov subj...
...,...,...
1194,from jerrybeskimocom jerry kaufman\nsubject re...,jerrybeskimocom jerry kaufman subject prayers ...
1195,from golchowyalchemychemutorontoca gerald olch...,golchowyalchemychemutorontoca gerald olchowy s...
1196,from jaynemmaltguildorg jayne kulikauskas\nsub...,jaynemmaltguildorg jayne kulikauskas subject q...
1197,from sclarkepasutorontoca susan clark\nsubject...,sclarkepasutorontoca susan clark subject picks...


## Latent Dirichlet Allocation model

👇 Train an LDA model to extract potential topics.

In [36]:
vectorizer = TfidfVectorizer().fit(data['clean_text'])
data_vectorized = vectorizer.transform(data['clean_text'])
lda_model = LatentDirichletAllocation(n_components = 2).fit(data_vectorized)

## Visualize potential topics

👇 The function to print the words associated with the potential topics is already made for you. You just have to pass the correct arguments!

In [37]:
def print_topics(model, vectorizer): 
    for idx, topic in enumerate (model.components_):
        print("Topic %d" %(idx))
        print([(vectorizer.get_feature_names()[i], topic[i])
            for i in topic.argsort()[:-10 -1 : -1]])

print_topics(lda_model, vectorizer)

Topic 0
[('bos', 3.0924912738025427), ('chi', 2.797943810494836), ('pit', 2.4077360253351867), ('det', 2.38142219862452), ('tor', 1.8893588144294724), ('buf', 1.8576178810419401), ('cal', 1.8531954285690289), ('que', 1.634100104859753), ('stl', 1.6267623388971522), ('nyi', 1.5834232077713832)]
Topic 1
[('god', 29.912972885875064), ('would', 25.80717670634672), ('one', 23.016584105516607), ('subject', 22.420375456587685), ('organization', 21.537672320806617), ('university', 21.473175880618513), ('lines', 21.46427500086195), ('writes', 20.397380678360175), ('people', 20.371659123305967), ('game', 19.557774220862203)]


In [38]:
clean_text_vectorized = vectorizer.transform(data['clean_text'])
lda_vector = lda_model.transform(clean_text_vectorized)

print("topic 0 :", lda_vector[0][0])
print("topic 1 :", lda_vector[0][1])

topic 0 : 0.049565961302878346
topic 1 : 0.9504340386971216


In [39]:
# vectorizer.get_params
# impact des 20 permiers tokens de la première matrice 
lda_model.components_[0][:20]

array([0.50398095, 0.50371152, 0.50481415, 0.5060452 , 0.50393756,
       0.50180376, 0.50637134, 0.50206863, 0.50297531, 0.503775  ,
       0.5031373 , 0.50084097, 0.50247568, 0.50443294, 0.50025716,
       0.50122351, 0.50092514, 0.50317737, 0.50342399, 0.50087219])

## Predict topic of new text

👇 You can now use your LDA model to predict the topic of a new text. First, use your vectorizer to vectorize the example. Then, use your LDA model to predict the topic of the vectorized example.

In [41]:
text= ["""Il y a des natures purement contemplatives et tout à fait impropres à l’action, 
qui cependant, sous une impulsion mystérieuse et inconnue, agissent quelquefois avec 
une rapidité dont elles se seraient crues elles-mêmes incapables."""]

In [44]:
vec = vectorizer.transform(text)
print(lda_model.transform(vec))

[[0.21866227 0.78133773]]


Le text semble appartenir à la catégorie 1.